# Processing text queries
- code in this notebook predominantly analyze text queries submitted by participants to solve KIS tasks of VBS 2023
- among others, this notebook can replicate the content of Table 2 and 3 as well as Figures 13-15

In [ ]:
import sys
import os
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from notebooks.utils import compute_user_penalty, get_team_values_df
from common.load import load_competition_data, process_team_logs

pd.set_option('display.max_colwidth', None)
unknownRankLimit = 1000
unknownRankValue = 2000

# Import common data

In [ ]:
config = 'config_vbs2023.yaml'

# load competition data from dres files and auxiliary data (FPSs, sequences)
comp_data = load_competition_data(config)

# load the preprocessed query data
dataset = pd.read_pickle(comp_data["config"]["processed_logs_outdir"] + '/text_query_dataset.pkl')

# valid teams
team_order = ['vibro', 'VISIONE',  'vitrivr-VR', 'CVHunter', 'Verge']
#team_order = ['vibro', 'VISIONE', 'VIREO' 'vitrivr-VR', 'CVHunter', 'vitrivr', 'Verge']

dataset.shape

### Creating auxiliary variables
- Query length and volume of words per query
- Maybe also store information whether the query is temporal? Only HTW and VISIONE have obviouse temporal queries
- Define visual vs textual tasks

In [ ]:
dataset["task_type"] = "visual"
dataset.loc[dataset.task.str.contains("kis-t"),"task_type"] = "textual"

dataset["query_type"] = "Other"
dataset.loc[(dataset.is_joint_embedding_text_query) &  ~(dataset.is_temporal_query), "query_type"] = "Text"
dataset.loc[(dataset.is_joint_embedding_text_query) &  (dataset.is_temporal_query), "query_type"] = "Text + Temporal"

dataset["QueryLen"] = -1
dataset["QueryWords"] = -1

dataset.loc[dataset["query_type"]=="Text","QueryLen"] = dataset.loc[dataset["query_type"]=="Text","value"].str.len()
dataset.loc[dataset["query_type"]=="Text","QueryWords"] = dataset.loc[dataset["query_type"]=="Text","value"].str.split().str.len()
dataset.loc[dataset["query_type"]=="Text + Temporal","QueryLen"] = dataset.loc[dataset["query_type"]=="Text + Temporal","value"].str.len()
dataset.loc[dataset["query_type"]=="Text + Temporal","QueryWords"] = dataset.loc[dataset["query_type"]=="Text + Temporal","value"].str.split().str.len()

dataset['user'] = dataset['user'].replace(0, '1st').replace(1, '2nd')

dataset.head()

In [ ]:
dataset.loc[(dataset['is_joint_embedding_text_query'] == False) & (dataset['team'] == 'VISIONE'), ["team", "category", "type", "query_type", "timestamp"]]

# Table 2 in the paper: usage of Text queries

In [ ]:
# percentage of how many rows per team belong to a text query and an image query
counts = dataset.groupby(["team","query_type"])[["value"]].count()
counts = ((counts / counts.groupby('team')[["value"]].sum())*100).round(1)

# add empty rows
counts = counts.reset_index()
counts = counts.append(pd.DataFrame([{'team': 'vitrivr-VR', 'query_type': 'Other', 'value': 0}], columns=counts.columns))

# sort in team order
counts = counts.set_index(['team','query_type']).loc[team_order, :]
counts

In [ ]:
print(counts.to_latex())

In [ ]:
queryCount = dataset.groupby(["team", "user", "query_type", "timestamp"])['task'].count().reset_index()
queryCount = queryCount.groupby(["team", "user", "query_type"]).count()
queryCount = queryCount['task']
queryCount

In [ ]:
queryCount.groupby(["team", "user"]).sum()

In [ ]:
((queryCount / queryCount.groupby(["team", "user"]).sum())*100).round(1)

# Table 3

In [ ]:
# median values
dataset.loc[dataset.rank_video > unknownRankLimit, "rank_video"] = unknownRankValue
dataset.loc[dataset.rank_shot_margin_0 > unknownRankLimit, "rank_shot_margin_0"] = unknownRankValue
dataset.loc[dataset.rank_shot_margin_5 > unknownRankLimit, "rank_shot_margin_5"] = unknownRankValue
medianStats = dataset.groupby(["team","user","query_type"])[["rank_shot_margin_0","rank_shot_margin_5","rank_video"]].median()

# mean query length and word count
meanQueryStats = dataset.groupby(["team","user","query_type"])[["QueryWords","QueryLen"]].mean()

# query count per team, user and query_type
queryCount = dataset.groupby(["team", "user", "query_type", "timestamp"])['task'].count().reset_index()
queryCount = queryCount.groupby(["team", "user", "query_type"]).count()
queryCount = queryCount['task']
#queryUsage = ((queryCount / dataset.groupby(["team", "user"])['task'].count())*100).round(1)
queryUsage = ((queryCount / queryCount.groupby(["team", "user"]).sum())*100).round(1)

# construct table
restrData = dataset
top5 = restrData.loc[restrData.rank_shot_margin_0 < 5].groupby(["team","user","query_type"]).count()["rank_shot_margin_0"]
top10 = restrData.loc[restrData.rank_shot_margin_0 < 10].groupby(["team","user","query_type"]).count()["rank_shot_margin_0"]
top20 = restrData.loc[restrData.rank_shot_margin_0 < 20].groupby(["team","user","query_type"]).count()["rank_shot_margin_0"]
top50 = restrData.loc[restrData.rank_shot_margin_0 < 50].groupby(["team","user","query_type"]).count()["rank_shot_margin_0"]
top100 = restrData.loc[restrData.rank_shot_margin_0 < 100].groupby(["team","user","query_type"]).count()["rank_shot_margin_0"]
top200 = restrData.loc[restrData.rank_shot_margin_0 < 200].groupby(["team","user","query_type"]).count()["rank_shot_margin_0"]
top500 = restrData.loc[restrData.rank_shot_margin_0 < 500].groupby(["team","user","query_type"]).count()["rank_shot_margin_0"]

resTab1 = pd.DataFrame({"total queries":queryCount,
                        "queries usage":queryUsage,
                        "words / query": 0,
                        "query length": 0,
                        "top5":(top5/queryCount*100).round(1),
                        "top10":(top10/queryCount*100).round(1),
                        "top20":(top20/queryCount*100).round(1),
                        "top50":(top50/queryCount*100).round(1),
                        "top100":(top100/queryCount*100).round(1),
                        "top200":(top200/queryCount*100).round(1),
                        "top500":(top500/queryCount*100).round(1)}).fillna(0)

# in the end we want mean stats
resTab1["words / query"] = meanQueryStats["QueryWords"]
resTab1["query length"] = meanQueryStats["QueryLen"]

# sort in team order
resTab1 = resTab1.loc[team_order, :]
resTab1

## add queries per minute to table 3

In [ ]:
# total availabe time to ask queries per team
totTime = dataset.groupby(["team", "task"])[["correct_submission_time_ms"]].min().reset_index().groupby(["team"])["correct_submission_time_ms"].sum()

# query per minute per user, team and query type
queryPerMinute = queryCount/totTime*1000*60
qpmdf = pd.DataFrame({"QPM": queryPerMinute})

# table 3
tab3 = resTab1.reset_index().set_index(['team', 'user', 'query_type'])

# copy queries per minute column for all clip rows
tab3["query / minute"] = qpmdf["QPM"]

# Replace NaN values with zeros
tab3 = tab3.fillna(0)
                   
# Replace -1 with 0
tab3 = tab3.replace(-1, 0)

# combine query count and usage columns
tab3["usage"] = tab3["total queries"].astype(str) + ' (' + tab3["queries usage"].astype(int).astype(str) + '%)'
                   
tab3[["usage", "query / minute","words / query","query length","top10","top20","top50","top100","top200"]].round(2)

## create empty rows to table 3

In [ ]:
from itertools import product

# Get unique values from each column
utab = tab3.reset_index()
unique_teams = utab['team'].unique()
unique_users = utab['user'].unique()
unique_query_types = utab['query_type'].unique()

# Generate all combinations
combinations = product(unique_teams, unique_users, unique_query_types)

# Create a new DataFrame from combinations
new_data = list(combinations)
new_df = pd.DataFrame(new_data, columns=['team', 'user', 'query_type'])

# Remove Verge second user
new_df = new_df[~((new_df['team'] == "Verge") & (new_df['user'] == "2nd"))]
new_df

In [ ]:
tab3full = new_df.set_index(['team', 'user', 'query_type'])

# Copy specific columns from df to new_df
columns_to_copy = ["usage", "query / minute","words / query","query length","top10","top20","top50","top100","top200"]
tab3full[columns_to_copy] = tab3[columns_to_copy]

# Replace NaN values with zeros
tab3full = tab3full.fillna(0)

# round
tab3full = tab3full.round(2)

# order
tab3full = tab3full.sort_values(by=['team', 'user', 'query_type'], ascending=[True, True, False]).loc[team_order, :]

# rename verge first user to first + second user
tab3full = tab3full.reset_index()
tab3full.loc[tab3full["team"] == "Verge", "user"] = "1st & 2nd"
tab3full = tab3full.set_index(['team', 'user', 'query_type'])

# add a percentage sign
columns_to_convert = ['top10', 'top20', 'top50', 'top100', 'top200']
tab3full[columns_to_convert] = tab3full[columns_to_convert].astype(str) + '%'

# replace 0 with -
tab3full = tab3full.replace(0, '-')
tab3full = tab3full.replace("0.0%", '-')

tab3full

In [ ]:
print(tab3full.to_latex())

# Figure 14

In [ ]:
textData = dataset.loc[dataset["query_type"]=="CLIP"]
textData["rankCat"] = ">100"
textData.loc[(textData.rank_shot_margin_0 <= 100),"rankCat"] = "1-100"

fig,ax = plt.subplots(1,1, figsize=(6,4))
textDataWithLength = textData.loc[textData.QueryLen > -1] # just to be sure
hue_order = team_order

#sns.boxenplot(data=textDataWithLength,x="rankCat",y="QueryWords", hue="team", ax=ax, hue_order = hue_order)
sns.boxenplot(data=textDataWithLength,x="rankCat",y="QueryLen", hue="team", ax=ax, hue_order = hue_order)

#ax.set_xlabel("Rank of the correct shot")
ax.set_xlabel("Rank of the correct shot")
#ax.set_ylabel("Words per query")
ax.set_ylabel("Query length")
plt.tight_layout()
plt.savefig("kis_TextQueryLengthVsRanking.pdf")